# Interactive LLM Agent Data Stream
- Prompt tuning+ HumanLoop
- WandB prompt tracing
- Autogen Agents
- Asyncio for async
- 


In [1]:
import os
import sys
import time  
import autogen
import wandb
import humanloop
import asyncio
from loguru import logger
from pathlib import Path
from datetime import datetime


from dotenv import load_dotenv
load_dotenv()
#config_list = autogen.config_list_from_json("OAI_CONFIG_LIST")


True

In [2]:
#CONSTANTS
PROJECT_NAME = input("Project Name: ")
MODEL = "gpt-3.5-turbo-0125"
PROMPT_TRACING = True
DEBUG = True  #All API Keys Exposed
LOG_LEVEL = DEBUG
#ENV = "PROD"
ENV = "DEV"

In [3]:
#Logging (Logoru)
if DEBUG == True:
    logger.add(sys.stderr, format=f"{datetime.date}", filter="my_module", level="DEBUG")
    logger.add(f"../logs/{datetime.now()}_{PROJECT_NAME}.log")
else:
    logger.add(sys.stderr, format=f"{datetime.date}", filter="my_module", level="INFO")
    logger.add(f"../logs/{datetime.now()}_{PROJECT_NAME}.log")
    

# User Config


In [ ]:
if Path("./user_config.yml").exists():
    config_list="./user_config.yml"
    logger.info("userconfig file found")
else
    config_list=
    

# Configuration

In [ ]:
if DEBUG == True:
    logger.info(f"Weights & Biases API Key: {os.getenv('WANDB_API_KEY')}")
    logger.info(f"HumanLoop API Key: {os.getenv('HUMANLOOP_API_KEY')}")
    logger.info(f"OpenAI API Key: {os.getenv('OPENAI_API_KEY')}")
    
    

# MLOPs / LLMOPS



In [ ]:

from humanloop import Humanloop


CUSTOM_HUMANLOOP_NAME = None

#Project Config
try:
    humanloop = Humanloop(api_key=os.getenv("HUMANLOOP_API_KEY"))
    project_response = humanloop.projects.create(name=PROJECT_NAME)
    project_id = project_response.body["id"]

    #Good/Bad etc
    #By default your project has a feedback type rating with labels good and bad.
    '''
    humanloop.projects.update_feedback_types(
        id=project_id,
        body=[{
            "type": "action", 
            "class": "multi_select", 
            "values": [
            {"value": "copied"}, {"value": "saved"}, {"value:": "deleted"}
            ],
        }]
    )'''

    #Register model config
    humanloop.model_configs.register(
        project=PROJECT_NAME,
        model=MODEL,
        prompt_template="You are a cybersecurity expert. Return detailed threat analyis for {{threat}}:",
        temperature=0.8,
    )
except Exception as e:
    logger.error(e)
    


In [ ]:
# HumanLoop Project
# humanloop.complete_deployed(...) will call the active model config on your project.
# The inputs must match the input of the prompt template in your project.
complete_response = humanloop.complete_deployed(
    project= "default_test",
    inputs={"question": "How should I think about competition for my startup?"},
    provider_api_keys={"openai": os.getenv("OPENAI_API_KEY"), "anthropic": os.getenv("CLAUDE_API_KEY")},
) 

# A single call to generate may return multiple outputs.
data_id = complete_response.body["data"][0]["id"]
output = complete_response.body["data"][0]["output"]

# You can also access the raw response from OpenAI.
print(complete_response.body["provider_responses"])

In [ ]:
LLM_PROMPTS = True
EXPERIMENT = False

try:     
    wandb.login(key=os.getenv("WANDB_API_KEY"))
except Exception as e:
    logger.error(f"Error: {e}")
    wandb.login()
    


In [ ]:
project=PROJECT_NAME
#Full list
wandb_config = {
     "learning_rate": 0.02,
    "architecture": "CNN",
    "dataset": "CIFAR-100",
    "epochs": 10,
    
}
wandb.init(project=PROJECT_NAME, config=wandb_config, entity="orionai")
# For WandB Prompts

#For Model Experimentation and tracking:




In [ ]:
#Log Metrics
 #wandb.log({"acc": acc, "loss": loss})

# Autogen Configuration

In [ ]:
# https://github.com/microsoft/autogen/blob/main/notebook/agentchat_stream.ipynb
config_list = autogen.config_list_from_json("OAI_CONFIG_LIST")


# Data Sources
- Simulating Stock Data but could do the same with CS

# Assistant Types
We have designed a generic ConversableAgent class for Agents that are capable of conversing with each other through the exchange of messages to jointly finish a task. An agent can communicate with other agents and perform actions. Different agents can differ in what actions they perform after receiving messages. Two representative subclasses are AssistantAgent and UserProxyAgent

The AssistantAgent is designed to act as an AI assistant, using LLMs by default but not requiring human input or code execution. It could write Python code (in a Python coding block) for a user to execute when a message (typically a description of a task that needs to be solved) is received. Under the hood, the Python code is written by LLM (e.g., GPT-4). It can also receive the execution results and suggest corrections or bug fixes. Its behavior can be altered by passing a new system message. The LLM inference configuration can be configured via [llm_config].

The UserProxyAgent is conceptually a proxy agent for humans, soliciting human input as the agent's reply at each interaction turn by default and also having the capability to execute code and call functions or tools. The UserProxyAgent triggers code execution automatically when it detects an executable code block in the received message and no human user input is provided. Code execution can be disabled by setting the code_execution_config parameter to False. LLM-based response is disabled by default. It can be enabled by setting llm_config to a dict corresponding to the inference configuration. When llm_config is set as a dictionary, UserProxyAgent can generate replies using an LLM when code execution is not performed.

The auto-reply capability of ConversableAgent allows for more autonomous multi-agent communication while retaining the possibility of human intervention. One can also easily extend it by registering reply functions with the register_reply() method.

In the following code, we create an AssistantAgent named "assistant" to serve as the assistant and a UserProxyAgent named "user_proxy" to serve as a proxy for the human user. We will later employ these two agents to solve a task.

In [ ]:
def get_market_news(ind, ind_upper):
    import json

    import requests

    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    # url = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=AAPL&sort=LATEST&limit=5&apikey=demo'
    # r = requests.get(url)
    # data = r.json()
    # with open('market_news_local.json', 'r') as file:
    #     # Load JSON data from file
    #     data = json.load(file)
    data = {
        "feed": [
            {
                "title": "Palantir CEO Says Our Generation's Atomic Bomb Could Be AI Weapon - And Arrive Sooner Than You Think - Palantir Technologies  ( NYSE:PLTR ) ",
                "summary": "Christopher Nolan's blockbuster movie \"Oppenheimer\" has reignited the public discourse surrounding the United States' use of an atomic bomb on Japan at the end of World War II.",
                "overall_sentiment_score": 0.009687,
            },
            {
                "title": '3 "Hedge Fund Hotels" Pulling into Support',
                "summary": "Institutional quality stocks have several benefits including high-liquidity, low beta, and a long runway. Strategist Andrew Rocco breaks down what investors should look for and pitches 3 ideas.",
                "banner_image": "https://staticx-tuner.zacks.com/images/articles/main/92/87.jpg",
                "overall_sentiment_score": 0.219747,
            },
            {
                "title": "PDFgear, Bringing a Completely-Free PDF Text Editing Feature",
                "summary": "LOS ANGELES, July 26, 2023 /PRNewswire/ -- PDFgear, a leading provider of PDF solutions, announced a piece of exciting news for everyone who works extensively with PDF documents.",
                "overall_sentiment_score": 0.360071,
            },
            {
                "title": "Researchers Pitch 'Immunizing' Images Against Deepfake Manipulation",
                "summary": "A team at MIT says injecting tiny disruptive bits of code can cause distorted deepfake images.",
                "overall_sentiment_score": -0.026894,
            },
            {
                "title": "Nvidia wins again - plus two more takeaways from this week's mega-cap earnings",
                "summary": "We made some key conclusions combing through quarterly results for Microsoft and Alphabet and listening to their conference calls with investors.",
                "overall_sentiment_score": 0.235177,
            },
        ]
    }
    feeds = data["feed"][ind:ind_upper]
    feeds_summary = "\n".join(
        [
            f"News summary: {f['title']}. {f['summary']} overall_sentiment_score: {f['overall_sentiment_score']}"
            for f in feeds
        ]
    )
    return feeds_summary


data = asyncio.Future()


async def add_stock_price_data():
    # simulating the data stream
    for i in range(0, 5, 1):
        latest_news = get_market_news(i, i + 1)
        if data.done():
            data.result().append(latest_news)
        else:
            data.set_result([latest_news])
        # print(data.result())
        await asyncio.sleep(5)


data_task = asyncio.create_task(add_stock_price_data())

# Cache

# Agents


In [ ]:
# create an AssistantAgent (AI Agent called "Threat Hunting")
threat_hunting = autogen.AssistantAgent(
    name="assistant",
    llm_config={
        "timeout": 600,
        "cache_seed": 41,
        "config_list": config_list,
        "temperature": 0,
    },
    system_message="You are a financial expert.",
)


In [ ]:
# create a UserProxyAgent instance named "user_proxy" which acts as human
user_proxy = autogen.UserProxyAgent(
    name="user",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=5,
    code_execution_config=False,
    default_auto_reply=None,
)

In [ ]:
# Use Redis as cache

with Cache.redis(redis_url="redis://localhost:6379/0") as cache:
    user.initiate_chat(threat_hunting, message=coding_task, cache=cache)

In [ ]:
async def add_data_reply(recipient, messages, sender, config):
    await asyncio.sleep(0.1)
    data = config["news_stream"]
    if data.done():
        result = data.result()
        if result:
            news_str = "\n".join(result)
            result.clear()
            return (
                True,
                f"Just got some latest market news. Merge your new suggestion with previous ones.\n{news_str}",
            )
        return False, None


user_proxy.register_reply(autogen.AssistantAgent, add_data_reply, position=2, config={"news_stream": data})

In [ ]:
await user_proxy.a_initiate_chat( 
    threat_hunting,
    message="""Give me investment suggestion in 3 bullet points.""",
)
while not data_task.done() and not data_task.cancelled():
    reply = await user_proxy.a_generate_reply(sender=threat_hunting)  # noqa: F704
    if reply is not None:
        await user_proxy.a_send(reply, threat_hunting)  # noqa: F704

In [ ]:
wandb.finish()